## Lab

Implement a URL fetcher using Beautiful Soup in the callback version. We will implement a similar one using coroutines on wednesday. 

The implimentation will extend the read_response method by parsing for URL's using `bs4` . Start by creating globals:
```
urls_todo = set(['/'])
seen_urls = set(['/'])
```

then:

```
links = self.parse_links()#write this
```
(using self.response)

Then use the set `difference` method  to add new links to `urls_todo` and recursively set up a `Fetcher` instance.

Now update the `seen_urls` and `urls_todo` thus:
```
seen_urls.update(links)
urls_todo.remove(self.url)
if not urls_todo:
    stopped = True
```

In [1]:
from bs4 import BeautifulSoup
import socket
from urllib.parse import urlparse

In [2]:
from selectors import DefaultSelector, EVENT_READ, EVENT_WRITE

selector = DefaultSelector()
class Fetcher:
    def __init__(self, host, url, level=0):
        self.response = b''  # Empty array of bytes.
        self.host = host
        self.url = url
        self.sock = None
        
    # Method on Fetcher class.
    def fetch(self):
        self.sock = socket.socket()
        self.sock.setblocking(False)
        try:
            self.sock.connect((self.host, 80))
        except BlockingIOError:
            pass

        # Register next callback.
        selector.register(self.sock.fileno(),
                          EVENT_WRITE,
                          self.connected)

    def connected(self, key, mask):
        print('connected!', flush=True)
        selector.unregister(key.fd)
        request = 'GET {} HTTP/1.0\r\nHost: {}\r\n\r\n'.format(self.url, self.host)
        self.sock.send(request.encode('ascii'))

        # Register the next callback.
        selector.register(key.fd,
                          EVENT_READ,
                          self.read_response)
        
    def read_response(self, key, mask):
        global stopped, urls_todo, seen_urls
        
        chunk = self.sock.recv(128)  # USUALLY 4k chunk size, here small
        if chunk:
            print("read chunk", flush=True)
            self.response += chunk
        else:
            print("all read", flush=True)
            selector.unregister(key.fd)  # Done reading.
            links = self.parse_links()
            links -= seen_urls
            if len(links) > 0:
                seen_urls.update(links)
                for url in links:
                    parsed_url = urlparse(url)
                    hostname = parsed_url.hostname
                    if hostname is None:
                        hostname = self.host
                    if parsed_url.path not in urls_todo:
                        urls_todo.add(parsed_url.path)
                        fetcher = Fetcher(hostname, parsed_url.path)
                        fetcher.fetch()
                urls_todo.remove(self.url)
                if not urls_todo:
                    stopped = True
            #This line will stop the reading, otherwise it'll go on without stopping
            else:
                stopped = True
            
    def parse_links(self):
        soup = BeautifulSoup(self.response, 'html.parser')
        links = set()
        for a in soup.find_all('a'):
            links.add(a.get('href'))
        return links
                    
stopped = False
urls_todo = set(['/'])
seen_urls = set(['/'])

def loop():
    while not stopped:
        events = selector.select()
        for event_key, event_mask in events:
            callback = event_key.data
            callback(event_key, event_mask)
        #do fibonacci

In [3]:
fetcher = Fetcher('xkcd.com', '/')
fetcher.fetch()
loop()

connected!
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
all read
connected!
connected!
connected!
connected!
connected!
connected!
connected!
connected!
connected!
connected!
connected!
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
all read
